# QUANTUM CONVOLUTIONAL NEURAL NETWORK

- 1 Dependencies
   - 1.1 Install the dependencies
   - 1.2 Load the dependencies
- 2 Data preprocessing
   - 2.1 Load the raw data
   - 2.2 Normalizing
   - 2.3 Filtering
   - 2.4 Resizing

- 3 Classical neural network
   - 3.1 Build the model
   - 3.2 Compile the model
   - 3.3 Train the model
   - 3.4 Evaluate the model

- 4 Quantum neural network
   - 4.1 Encode the data as quantum circuits
   - 4.2 Convert the quantum circuits to tensors
   - 4.3 Convert the quantum tensors to quantum circuits 
   - 4.4 Build the model
   - 4.5 Compile the model
   - 4.6 Train the model
   - 4.7 Evaluate the model
   
- 5 Comparison
   - 5.1 Create a dataframe
   - 5.2 Quantum CNN accuracy Vs Classical CNN accuracy
   - 5.3 Quantum CNN loss Vs Classical CNN loss
   - 5.4 Quantum Vs Classical using Barplot
   - 5.5 Performance Metrics
   - 5.6 Confusion Matrix


## 1. Dependencies

### 1.1 Install the dependencies

In [ ]:
!pip install tensorflow==2.7.0 tensorflow-quantum==0.7.2


### 1.2 Loads the dependencies

In [ ]:
import importlib, pkg_resources
importlib.reload(pkg_resources)


In [ ]:
import cirq
import sympy
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_quantum as tfq
from cirq.contrib.svg import SVGCircuit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,f1_score

## 2. Data preprocessing

### 2.1 Load the raw data

Load the MNIST dataset distributed with Keras. 

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

### 2.2 Normalizing

 Rescale the images from [0,255] to the [0.0,1.0] range.

In [ ]:
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))

### 2.3 Filtering

Filter the dataset to keep just the 3s and 6s,  remove the other classes. At the same time convert the label, `y`, to boolean: `True` for `3` and `False` for 6. 

In [ ]:
def filter_36(x, y):
    keep = (y == 3) | (y == 6)
    x, y = x[keep], y[keep]
    y = y == 3
    return x,y

In [ ]:
x_train, y_train = filter_36(x_train, y_train)
x_test, y_test = filter_36(x_test, y_test)

print("Number of filtered training examples:", len(x_train))
print("Number of filtered test examples:", len(x_test))

### 2.4 Resizing

An image size of 28x28 is much too large for current quantum computers. Resize the image down to 4x4:

In [ ]:
x_train_small = tf.image.resize(x_train, (4,4)).numpy()
x_test_small = tf.image.resize(x_test, (4,4)).numpy()

## 3. Classical neural network

### 3.1 Build the model 

In [ ]:
def create_classical_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, [3, 3], activation='relu', input_shape=(28,28,1)))
    model.add(tf.keras.layers.Conv2D(64, [3, 3], activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1))
    return model


model_cnn = create_classical_model()

### 3.2 Compile the model 

In [ ]:
model_cnn.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

### 3.3 Train the  model

Hyperpermeters

In [ ]:
EPOCHS = 2

In [ ]:
cnn_history = model_cnn.fit(x_train,y_train,batch_size=128,epochs=EPOCHS, verbose=1, validation_data=(x_test, y_test))

### 3.4 Evaluate the  model

In [ ]:
cnn_results = model_cnn.evaluate(x_test, y_test)

## 4. Quantum neural network



### 4.1 Encode the data as quantum circuits

In [ ]:
THRESHOLD = 0.5

x_train_bin = np.array(x_train_small > THRESHOLD, dtype=np.float32)
x_test_bin = np.array(x_test_small > THRESHOLD, dtype=np.float32)

### 4.2 Convert the quantum circuits to tensors


In [ ]:
def convert_to_circuit(image):
    values = np.ndarray.flatten(image)
    qubits = cirq.GridQubit.rect(4, 4)
    circuit = cirq.Circuit()
    for i, value in enumerate(values):
        if value:
            circuit.append(cirq.X(qubits[i]))
    return circuit


In [ ]:
x_train_circ = [convert_to_circuit(x) for x in x_train_bin]
x_test_circ = [convert_to_circuit(x) for x in x_test_bin]

### 4.3 Convert the quantum tensors to quantum circuits 

In [ ]:
x_train_tfcirc = tfq.convert_to_tensor(x_train_circ)
x_test_tfcirc = tfq.convert_to_tensor(x_test_circ)

### 4.4 Build the model 


In [ ]:
class CircuitLayerBuilder():
    def __init__(self, data_qubits, readout):
        self.data_qubits = data_qubits
        self.readout = readout
    
    def add_layer(self, circuit, gate, prefix):
        for i, qubit in enumerate(self.data_qubits):
            symbol = sympy.Symbol(prefix + '-' + str(i))
            circuit.append(gate(qubit, self.readout)**symbol)

In [ ]:
def create_quantum_model():
    data_qubits = cirq.GridQubit.rect(4, 4) 
    readout = cirq.GridQubit(-1, -1)        
    circuit = cirq.Circuit()
    circuit.append(cirq.X(readout))
    circuit.append(cirq.H(readout))
    builder = CircuitLayerBuilder(data_qubits = data_qubits,readout=readout)
    builder.add_layer(circuit, cirq.XX, "xx1")
    builder.add_layer(circuit, cirq.ZZ, "zz1")
    circuit.append(cirq.H(readout))

    return circuit, cirq.Z(readout)

In [ ]:
model_circuit, model_readout = create_quantum_model()

In [ ]:
model_qcnn = tf.keras.Sequential([tf.keras.layers.Input(shape=(), dtype=tf.string),tfq.layers.PQC(model_circuit, model_readout),])

### 4.5 Compile the model 

In [ ]:
y_train_hinge = 2.0*y_train-1.0
y_test_hinge = 2.0*y_test-1.0

In [ ]:
def hinge_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true) > 0.0
    y_pred = tf.squeeze(y_pred) > 0.0
    result = tf.cast(y_true == y_pred, tf.float32)

    return tf.reduce_mean(result)

In [ ]:
model_qcnn.compile(loss=tf.keras.losses.Hinge(),optimizer=tf.keras.optimizers.Adam(),metrics=[hinge_accuracy])

### 4.6 Train the model

Hyperpermeters

In [ ]:
NUM_EXAMPLES = len(x_train_tfcirc)
x_train_tfcirc_sub = x_train_tfcirc[:NUM_EXAMPLES]
y_train_hinge_sub = y_train_hinge[:NUM_EXAMPLES]

In [ ]:
qnn_history = model_qcnn.fit(x_train_tfcirc_sub, y_train_hinge_sub,batch_size=32,epochs=EPOCHS,verbose=1,validation_data=(x_test_tfcirc, y_test_hinge))

### 4.7 Evaluate the  model

In [ ]:
qnn_results = model_qcnn.evaluate(x_test_tfcirc, y_test)

## 5. Comparison

### 5.1 Create a dataframe

In [ ]:
qnn_history.history
accuracy = pd.DataFrame()
loss = pd.DataFrame()
accuracy['QNN_accuracy']  = qnn_history.history['hinge_accuracy']
accuracy['QNN_val_accuracy'] = qnn_history.history['val_hinge_accuracy']
accuracy['CNN_accuracy']  = cnn_history.history['accuracy']
accuracy['CNN_val_accuracy'] = cnn_history.history['val_accuracy']
loss['QNN_loss']      = qnn_history.history['loss']
loss['QNN_val_loss']  = qnn_history.history['val_loss']
loss['CNN_loss']      = cnn_history.history['loss']
loss['CNN_val_loss']  = cnn_history.history['val_loss']
print("Quantum CNN accuracy and  Classical CNN accuracy\n")
print(accuracy)
print("\nQuantum CNN loss and  Classical CNN loss\n")
print(loss)

### 5.2 Quantum CNN accuracy Vs Classical CNN accuracy

In [ ]:
def plot_accuracy():
    plt.plot(accuracy['QNN_accuracy'],'b--',label = 'QNN_accuracy')
    plt.plot(accuracy['QNN_val_accuracy'],'r--',label = 'QNN_val_accuracy')
    plt.plot(accuracy['CNN_accuracy'],'g--',label = 'CNN_accuracy')
    plt.plot(accuracy['CNN_val_accuracy'],'y--',label = 'CNN_val_accuracy')
    plt.title('Quantum CNN accuracy Vs Classical CNN accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.show()

In [ ]:
plot_accuracy()

### 5.3 Quantum CNN loss Vs  Classical CNN loss

In [ ]:
def plot_loss():
    plt.plot(loss['QNN_loss'],'b--',label = 'QNN_loss')
    plt.plot(loss['QNN_val_loss'],'r--',label = 'QNN_val_loss')
    plt.plot(loss['CNN_loss'],'g--',label = 'CNN_loss')
    plt.plot(loss['CNN_val_loss'],'y--',label = 'CNN_val_loss')
    plt.legend()
    plt.title('Quantum CNN loss Vs Classical CNN loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

In [ ]:
plot_loss()

### 5.4 Quantum Vs Classical using Barplot 

In [ ]:
accuracy =[qnn_results[1],cnn_results[1]]
lable = ["Quantum", "Classical"]
barlist = plt.bar(lable,accuracy)
barlist[0].set_color('b')
barlist[1].set_color('r')
plt.title("Quantum Vs Classical")
plt.show()

### 5.5 Performance Metrics

In [ ]:
def metrices_cnn(model):
    cnn_results = model.evaluate(x_test, y_test)
    precision = precision_score(y_test, model.predict(x_test) > 0)
    recall = recall_score(y_test, model.predict(x_test) > 0)
    f1 = f1_score(y_test, model.predict(x_test) > 0)
    metrices = {"accuracy" : cnn_results[1],"precision":precision,"recall_score":recall,"f1_score":f1,"loss":cnn_results[0]}
    return metrices

In [ ]:
def metrices_qcnn(model):
    qnn_predictions = model.predict(x_test_tfcirc)
    qnn_predictions = np.squeeze(qnn_predictions) > 0.0
    qnn_predictions = qnn_predictions.astype(int)
    acc = accuracy_score(y_test, qnn_predictions)
    precision = precision_score(y_test, qnn_predictions)
    recall = recall_score(y_test,qnn_predictions)
    f1 = f1_score(y_test, qnn_predictions)
    metrices = {"accuracy" : acc ,"precision":precision,"recall_score":recall,"f1_score":f1,"Loss":qnn_history.history['loss'][1]}
    return metrices

In [ ]:
metrices_cnn(model_cnn)

In [ ]:
metrices_qcnn(model_qcnn)

### 5.6 Confusion Matrix

In [ ]:
def draw_cm_qcnn(model):
    qnn_predictions = model.predict(x_test_tfcirc)
    qnn_predictions = np.squeeze(qnn_predictions) > 0.0
    qnn_predictions = qnn_predictions.astype(int)
    qnn_cm = confusion_matrix(y_test, qnn_predictions)
    qnn_cm_df = pd.DataFrame(qnn_cm, index=['3', '6'], columns=['3', '6'])
    plt.figure(figsize=(12, 6))
    sns.heatmap(qnn_cm_df, annot=True, fmt="d", cmap="Blues")
    plt.title('QCNN Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('actual Labels')
    plt.show()


In [ ]:
def draw_cm_cnn(model):
    y_pred = model.predict(x_test) > 0
    cm = confusion_matrix(y_test, y_pred)
    cm = pd.DataFrame(cm, index=['3', '6'], columns=['3', '6'])
    cm_sns = sns.heatmap(cm, cmap="Blues", annot=True ,fmt='g')
    plt.title('CNN Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('actual Labels')
    plt.show()

In [ ]:
draw_cm_cnn(model_cnn)

In [ ]:
draw_cm_qcnn(model_qcnn)